# METAR data on nearest airport
https://ourairports.com/data/

https://aviationweather.gov/dataserver/example?datatype=metar

https://aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&startTime=2023-01-21T19:09:20Z&endTime=2023-01-21T21:09:20Z&stationString=PHTO

In [4]:
# pip install beautifulsoup4
# pip install lxml

In [5]:
import requests
import json
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [6]:
### Load airports data
airports = pd.read_csv('airports.csv')
airports = airports[airports.scheduled_service == 'yes'] # only aiprt with weather station

In [48]:
def haversine(lon1, lat1, lon2, lat2):
    '''
    Calculate the great circle distance in nautical miles between two points 
    on the earth
    '''
    # Radius of earth in nm (6371 for miles)
    r = 3956 
    
    # degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return 2 * np.arcsin(np.sqrt(a))  * r

def request_metars(start_date,end_date,oaci):
    '''
    Read xml file from https://aviationweather.gov/dataserver/example?datatype=metar API
    and extract METARS.
    sd: start date
    sh: start hour
    ed: end date
    eh: en hour
    returns a pandas dataframe with METAR, temperature, visibility and flight rule
    '''
    sd, sh = start_date.split()[0], start_date.split()[1]
    ed, eh = end_date.split()[0], end_date.split()[1]
    
    url = "https://aviationweather.gov/adds/dataserver_current/httpparam?"\
        + "dataSource=metars&requestType=retrieve&format=xml&"\
        +"startTime={}T{}Z&endTime={}T{}Z&stationString={}".format(sd,sh,ed,eh,oaci)
    
    response = requests.get(url)
    bs_data = BeautifulSoup(response.text, "xml")
    #print(bs_data.prettify()) # uncomment to see all xml file
    
    dict_METAR = {}
    for tag in ['raw_text','temp_c','visibility_statute_mi','flight_category']:
        dict_METAR[tag] = []
        for link in bs_data.find_all(tag):
            dict_METAR[tag].append(link.contents[0]) 

    results = pd.DataFrame(dict_METAR)
    results['temp_c'] = results['temp_c'].astype(float)
    results['visibility_statute_mi'] = results['visibility_statute_mi'].astype(float)
    
    return results

def get_nearest_apt_metar(lat, lon, start_date, end_date, tries = 20):
    '''
    Get METARS from the nearest airport of the lat, lon point during a time range
    between start_date and end_date. 
    tries: number of OACI requests by increasing distance to lat, lon point
    '''
    
    # Get the nearest airport with weather service
    best_match = airports.iloc[
        (abs(airports['latitude_deg']-lat) + abs(airports['longitude_deg']-lon)).argsort()
    ].head(tries)

    best_match['distance'] = best_match.apply(lambda x: haversine(lon, lat, x.longitude_deg, x.latitude_deg), axis=1)
    best_match = best_match.reset_index(drop=True)

    # Search valid metars in the 20th nearest aiprorts during the specified time range
    for idx in range(best_match.shape[0]):
        
        best_match.iloc[idx].ident
        df_metar = request_metars(start_date,end_date,best_match.iloc[idx].ident)
        
        if df_metar.shape[0] > 0:
            df_metar['distance'] = best_match.iloc[idx].distance
            return df_metar
        
    return 'no metar found'

In [49]:
lat, lon = 48.403088, 11.517864
start_date = '2023-01-21 19:09:20'
end_date = '2023-01-21 21:09:20'

get_nearest_apt_metar(lat, lon, start_date, end_date)

,raw_text,temp_c,visibility_statute_mi,flight_category,distance
0,EDDM 212050Z AUTO 32006KT 9999 FEW010 BKN044 M...,-2.0,6.21,VFR,12.763399
1,EDDM 212020Z AUTO 32004KT 9999 -SN FEW010 BKN0...,-2.0,6.21,MVFR,12.763399
2,EDDM 211950Z AUTO 33005KT 9000 -SN SCT015 BKN0...,-2.0,5.59,VFR,12.763399
3,EDDM 211920Z AUTO 33006KT 290V350 6000 -SN FEW...,-2.0,3.73,MVFR,12.763399


'EDDM'